In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/CNN
!ls

/content/gdrive/My Drive/CNN
 1-conv-128-nodes-0-dense-1681324451-model-0.6954314708709717-accuracy.h5
 1-conv-128-nodes-0-dense-1681351857-model-0.7284263968467712-accuracy.h5
 1-conv-128-nodes-1-dense-1681337863-model-0.4517766535282135-accuracy.h5
 1-conv-32-nodes-0-dense-1681319480-model-0.7081218361854553-accuracy.h5
 1-conv-32-nodes-0-dense-1681346471-model-0.692893385887146-accuracy.h5
 1-conv-32-nodes-1-dense-1681332123-model-0.25380709767341614-accuracy.h5
 1-conv-32-nodes-1-dense-1681360383-model-0.1878172606229782-accuracy.h5
 1-conv-64-nodes-0-dense-1681321216-model-0.703045666217804-accuracy.h5
 1-conv-64-nodes-0-dense-1681348356-model-0.710659921169281-accuracy.h5
 1-conv-64-nodes-1-dense-1681333654-model-0.39593908190727234-accuracy.h5
 1-conv-64-nodes-1-dense-1681362187-model-0.1928934007883072-accuracy.h5
 2-conv-128-nodes-0-dense-1681325573-model-0.7055837512016296-accuracy.h5
 2-conv-128-nodes-0-dense-1681353036-model-0.7131979465484619-accuracy.h5
 2-conv-128-nodes-

In [3]:
%tensorflow_version 

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
import pickle
import numpy as np
import tensorflow as tf
pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

IMG_SIZE = (224, 224)
# Resize the training images
X_train = tf.image.resize(X_train, IMG_SIZE)

# Resize the validation images
X_test = tf.image.resize(X_test, IMG_SIZE)

# X_train = np.array(X_train)
# X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [5]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg16
import keras.layers as layers
from keras.layers import Conv2D
import numpy as np

In [6]:
tensorboard = TensorBoard(log_dir='./VGG16logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [7]:
vgg16_model = vgg16.VGG16()
vgg16_model.summary()

type(vgg16_model)

553467096/553467096 [==============================] - 24s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

keras.engine.functional.Functional

In [8]:
model1 = Sequential()

#for layer in vgg16_model.layers:
for layer in vgg16_model.layers[:-1]: #model1.layers.pop() not working, so -1
    model1.add(layer) # convert to sequantial model
    
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [9]:
for layer in model1.layers:
    layer.trainable = False
    
model1.add(Dense(4,activation = 'softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [10]:
# model1.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
model1.compile(loss='sparse_categorical_crossentropy',optimizer= "adam", metrics=['accuracy'],)

In [11]:
#Fit the model

history = model1.fit(X_train, Y_train,
            batch_size=32,
            epochs=20,
            validation_data=(X_test,Y_test),
            callbacks=[es])

Epoch 1/20
90/90 [==============================] - 28s 191ms/step - loss: 0.7145 - accuracy: 0.7383 - val_loss: 1.8997 - val_accuracy: 0.5381
Epoch 2/20
90/90 [==============================] - 14s 152ms/step - loss: 0.3744 - accuracy: 0.8599 - val_loss: 1.5597 - val_accuracy: 0.6853
Epoch 3/20
90/90 [==============================] - 13s 142ms/step - loss: 0.3145 - accuracy: 0.8836 - val_loss: 1.5294 - val_accuracy: 0.6853
Epoch 4/20
90/90 [==============================] - 13s 142ms/step - loss: 0.2388 - accuracy: 0.9139 - val_loss: 1.5919 - val_accuracy: 0.6929
Epoch 5/20
90/90 [==============================] - 14s 154ms/step - loss: 0.2158 - accuracy: 0.9237 - val_loss: 1.4428 - val_accuracy: 0.7005
Epoch 6/20
90/90 [==============================] - 13s 144ms/step - loss: 0.1818 - accuracy: 0.9359 - val_loss: 1.6333 - val_accuracy: 0.7487
Epoch 7/20
90/90 [==============================] - 14s 157ms/step - loss: 0.1627 - accuracy: 0.9453 - val_loss: 1.7818 - val_accuracy: 0.7386

In [12]:
#Score 
scores = model1.evaluate(X_test, Y_test, verbose=1) 
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

13/13 [==============================] - 2s 118ms/step - loss: 1.7006 - accuracy: 0.7614
Test loss: 1.700623631477356
Test accuracy: 0.7614213228225708


In [13]:
model1.save("VGG16model94accuracy.h5")

In [14]:
 #precision    recall  f1-score   support
y_pred = model1.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

7/7 [==============================] - 10s 219ms/step
              precision    recall  f1-score   support

           0       1.00      0.29      0.45       100
           1       0.70      0.98      0.82       115
           2       0.75      1.00      0.86       105
           3       0.91      0.78      0.84        74

    accuracy                           0.77       394
   macro avg       0.84      0.76      0.74       394
weighted avg       0.83      0.77      0.74       394

